# en este archivo cree el sistema de recomendacion pero despues de hacer ciertos cambios no me funciona correrlo aqui pero aun funciona en el archivo que se llama por la api

In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel # calcular la similitud coseno entre los juegos

## cargamos el dataframe con item_id y genres

In [3]:
df_juegos = pd.read_parquet(r"../app/df_modelo.parquet")

    vemos que se me pasaron 2 nulos que removere para empezar con el medelo

In [4]:
df_juegos.isna().sum()

genres      0
app_name    1
id          1
dtype: int64

In [5]:
df_juegos.dropna(inplace=True)

In [6]:
df_juegos.isna().sum()

genres      0
app_name    0
id          0
dtype: int64

    visuaalisamos el df_juegos

In [7]:
df_juegos
#el archivo tiene 32k de filas y es muy pesado para pasarle todos los datos

,genres,app_name,id
88310,Action Casual Indie Simulation Strategy,Lost Summoner Kitty,761140.0
88311,Free to Play Indie RPG Strategy,Ironbound,643980.0
88312,Casual Free to Play Indie Simulation Sports,Real Pool 3D - Poolians,670290.0
88313,Action Adventure Casual,弹炸人2222,767400.0
88314,Indie Sports Casual Action,Log Challenge,773570.0
...,...,...,...
120440,Casual Indie Simulation Strategy,Colony On Mars,773640.0
120441,Casual Indie Strategy,LOGistICAL: South Africa,733530.0
120442,Indie Racing Simulation,Russian Roads,610660.0
120443,Casual Indie,EXIT 2 - Directions,658870.0


    incializacion  de objeto tfidfvectorizer

In [8]:
vectorizer = TfidfVectorizer(stop_words='english',max_features=5000)
#stop_words parametro paraque ignore las palabras comunes de el ingles como (and,the,to) ya que el tfid vectorizer va a sacar el peso de cada palabra individual y tenemos algunos como free to play y pueden haber mas 
#max_features fue mi primer intento de reducir el espacio en memoria que ocupaba la matriz #redujo casi 3/4 de lo que ocupaba de memoria

    tomo una muestra aleatoria de los datos

In [9]:
df_muestra = df_juegos.sample(frac=0.1,random_state=42)#tomo una muestra de 10 % de el dataframe

In [10]:
df_muestra

,genres,app_name,id
112656,Strategy,Deserts of Kharak Expedition Guide,433240.0
107608,Education,Air Rage,583510.0
94843,Action,Furi,423230.0
113560,Action Indie Strategy,Awesomenauts - Wicked Coco Skin,410100.0
119404,Action Indie Simulation,SOL: Exodus,200410.0
...,...,...,...
110438,Adventure Indie,Rising Islands,452450.0
107010,Indie RPG Strategy Early Access,Wizards and Warlords,567080.0
101572,Action Adventure Simulation Early Access,Live And Learn,757900.0
110748,Casual,Evolution Planet: Gold Edition,495090.0


    vuelvo a crear los indices

In [11]:
df_muestra = df_muestra.reset_index(drop=True)

In [12]:
df_muestra

,genres,app_name,id
0,Strategy,Deserts of Kharak Expedition Guide,433240.0
1,Education,Air Rage,583510.0
2,Action,Furi,423230.0
3,Action Indie Strategy,Awesomenauts - Wicked Coco Skin,410100.0
4,Action Indie Simulation,SOL: Exodus,200410.0
...,...,...,...
3208,Adventure Indie,Rising Islands,452450.0
3209,Indie RPG Strategy Early Access,Wizards and Warlords,567080.0
3210,Action Adventure Simulation Early Access,Live And Learn,757900.0
3211,Casual,Evolution Planet: Gold Edition,495090.0


<hr>

    creando matriz_tfidf con la columna genres(generos) de la muestra

In [13]:
matriz_tfidf = vectorizer.fit_transform(df_muestra["genres"])

    calculando la similitud entre todas las columnas de la matriz de generos

In [14]:
similitud_juegos = linear_kernel(matriz_tfidf, matriz_tfidf) #calcula la similitud entre los juegos a partir de la matriz tfidf

    funcion para obtener recomendaciones

In [26]:
def obtener_recomendaciones(item_id_ingresado, n=5):
    try:
        item_id_ingresado = float(item_id_ingresado)
        indice_juego_ingresado = df_muestra[df_muestra['id'] == item_id_ingresado].index[0]
    except IndexError:
        return f"El juego con ID {int(item_id_ingresado)} no está en el conjunto de datos de entrenamiento."

    similitudes = similitud_juegos[indice_juego_ingresado]

    # Obtener los índices de los juegos más similares
    indices_similares = similitudes.argsort()[::-1][1:n+1]

    # Obtener los item_id de los juegos recomendados
    juegos_recomendados = df_muestra.loc[indices_similares, 'app_name'].tolist()

    return str({"recomendaciones": juegos_recomendados})

In [50]:
#intento de lograr que cuando se le pase un item que no se encuentra en la muestra se busque en el dataframe completo(df_juegos) y si se encuentra se agrege a la muestra  para volver a crear la matriz tfidf y volver a calcular la similitud con linear kernel
#pero al parecer no solo no funciona si no que tambien vacia los dataframe que estamos usando y hay que volver a cargar todo
#no voy a intentar seguir con  esto por que eh de seguir completando el eda e intentar hacer laa recomendacion de  usuario pero esta vez de una manera distinta 

#def obtener_recomendaciones(item_id_ingresado, n=5):
#    try:
#        item_id_ingresado = float(item_id_ingresado)
#        indice_juego_ingresado = df_muestra[df_muestra['id'] == item_id_ingresado].index[0]
#
#    except IndexError:
#        nuevo_item = df_juegos[df_juegos["id"] == item_id_ingresado]
#
#
#        if not nuevo_item.empty:
#            df_muestra = pd.concat([df_muestra, nuevo_item], ignore_index=True)
#
#            matriz_tfidf = vectorizer.fit_transform(df_muestra["genres"])
#
#            try:
#                indice_juego_ingresado = df_muestra[df_muestra['id'] == item_id_ingresado].index[0]
#
#                similitud_juegos = linear_kernel(matriz_tfidf, matriz_tfidf) #calcula la similitud entre los juegos a partir de la nueva matriz tfidf
#                similitudes = similitud_juegos[indice_juego_ingresado]
#
#                # Obtener los índices de los juegos más similares
#                indices_similares = similitudes.argsort()[::-1][1:n+1]#empieza desde el 1 hastas el 6 sin incluirlo
#
#                # Obtener los item_id de los juegos recomendados
#                juegos_recomendados = df_muestra.loc[indices_similares, 'app_name'].tolist()
#
#                return juegos_recomendados
#            except IndexError:
#                return f"El juego con ID {int(item_id_ingresado)} no existe en el datasets de steam."
#
#    similitudes = similitud_juegos[indice_juego_ingresado]
#
#    # Obtener los índices de los juegos más similares
#    indices_similares = similitudes.argsort()[::-1][1:n+1]#empieza desde el 1 hastas el 6 sin incluirlo
#
#    # Obtener los item_id de los juegos recomendados
#    juegos_recomendados = df_muestra.loc[indices_similares, 'app_name'].tolist()
#
#    return juegos_recomendados

In [27]:
recomendaciones = obtener_recomendaciones(295201, n=5)
print(recomendaciones)

El juego con ID 295201 no está en el conjunto de datos de entrenamiento.


In [25]:
obtener_recomendaciones(355250, n=5) #  star wars

'El juego con ID 355250 no está en el conjunto de datos de entrenamiento.'

    #el juego que ingresamos es star wars
    nos recomienda naruto shippuden juego de accion trata sobre la 4 guerra ninja
    golden kagura pigtail juego de accion
    psychonauts juego de accion y aventura
    call of duty  un juego de accion ,disparos ,guerra
    the little war juego de guerra por lo que se lee en el nombre

<hr>

In [126]:
df_juegos[df_juegos["id"] == 355250]


,genres,app_name,id
115227,Action,STAR WARS™: TIE Fighter Special Edition,355250.0


In [ ]:
df_muestra.aa